### 1. Import Dependencies

In [79]:
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFaceHub
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

load_dotenv();

### 2. Read PDF file

In [80]:
a = [1,2,3]
list("document\yolo.pdf".split(" "))

['document\\yolo.pdf']

In [54]:
type("a")

str

In [55]:
def read_multiple_pdf(files):
    if type(files) == str:
        files = list("document\yolo.pdf".split(" "))
    texts = ""
    for file in files:
        docs = PdfReader(file)
        for text in docs.pages:
            texts += (text.extract_text())
        return texts

In [56]:
file_path = "document\yolo.pdf"
docs = read_multiple_pdf(file_path)
len(docs), docs[:100]

(42084,
 'You Only Look Once:\nUniﬁed, Real-Time Object Detection\nJoseph Redmon\x03, Santosh Divvala\x03y, Ross Girsh')

### 3. Split PDF into Chunks

In [4]:
def chunk_docs(document, chunk_size = 500, chunk_overlap = 50, separators="\n"):
    text_splitter = RecursiveCharacterTextSplitter(
        separators=separators,
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap
    )
    chunk = text_splitter.split_text(document)
    return chunk

In [5]:
chunks = chunk_docs(docs)
len(chunks), chunks[:5]

(91,
 ['You Only Look Once:\nUniﬁed, Real-Time Object Detection\nJoseph Redmon\x03, Santosh Divvala\x03y, Ross Girshick{, Ali Farhadi\x03y\nUniversity of Washington\x03, Allen Institute for AIy, Facebook AI Research{\nhttp://pjreddie.com/yolo/\nAbstract\nWe present YOLO, a new approach to object detection.\nPrior work on object detection repurposes classiﬁers to per-\nform detection. Instead, we frame object detection as a re-\ngression problem to spatially separated bounding boxes and',
  'associated class probabilities. A single neural network pre-\ndicts bounding boxes and class probabilities directly from\nfull images in one evaluation. Since the whole detection\npipeline is a single network, it can be optimized end-to-end\ndirectly on detection performance.\nOur uniﬁed architecture is extremely fast. Our base\nYOLO model processes images in real-time at 45 frames\nper second. A smaller version of the network, Fast YOLO,\nprocesses an astounding 155 frames per second while',
  'sti

### 4. Embeds the data

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")
print(f"The Dimension of embedding: {len(embeddings.embed_query('Hello World'))}")

d:\Code Workspace\Personal Project\Personal Project - LangChain Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The Dimension of embedding: 384


In [27]:
def embedding_chunks(chunk, model_name = "sentence-transformers/all-MiniLM-L12-v2"):
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    vector_stores = FAISS.from_texts(chunks, embeddings)
    return vector_stores

In [28]:
vector_stores = embedding_chunks(chunks)

In [29]:
query = "What is YOLO?"
result = vector_stores.similarity_search(query, k= 10)
result

[Document(page_content='other real-time systems. For a demo of our system running\nin real-time on a webcam please see our project webpage:\nhttp://pjreddie.com/yolo/ .\nSecond, YOLO reasons globally about the image when\n1arXiv:1506.02640v5  [cs.CV]  9 May 2016making predictions. Unlike sliding window and region\nproposal-based techniques, YOLO sees the entire image\nduring training and test time so it implicitly encodes contex-\ntual information about classes as well as their appearance.'),
 Document(page_content='a small beneﬁt while YOLO provides a signiﬁcant performance\nboost.VOC 2012 test mAP aero bike bird boat bottle bus car cat chair cow table dog horse mbike personplant sheep sofa train tv\nMR CNN MORE DATA [11] 73.9 85.5 82.9 76.6 57.8 62.7 79.4 77.2 86.6 55.0 79.1 62.2 87.0 83.4 84.7 78.9 45.3 73.4 65.8 80.3 74.0\nHyperNet VGG 71.4 84.2 78.5 73.6 55.6 53.7 78.7 79.8 87.7 49.6 74.9 52.1 86.0 81.7 83.3 81.8 48.6 73.5 59.4 79.9 65.7'),
 Document(page_content='and height direc

### 5. Query the Docs to get the Answer Back

In [25]:
config = {
    'max_new_tokens': 256, 
    'temperature': 0.6,
}

llm = HuggingFaceHub(
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",
    model_kwargs = config
)

d:\Code Workspace\Personal Project\Personal Project - LangChain Project\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [70]:
memory = ConversationBufferMemory(
    memory_key= "chat_history",
    return_messages=True
)

In [71]:
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='hi'), AIMessage(content='whats up')]}

In [81]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever= vector_stores.as_retriever(),
    memory = memory
)

In [82]:
conversation_chain

ConversationalRetrievalChain(memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='whats up')]), return_messages=True, memory_key='chat_history'), combine_docs_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=HuggingFaceHub(client=InferenceAPI(api_url='https://api-inference.huggingface.co/pipeline/text-generation/mistralai/Mixtral-8x7B-Instruct-v0.1', task='text-generation', options={'wait_for_model': True, 'use_gpu': False}), repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1', model_kwargs={'max_new_tokens': 256, 'temperature': 0.6})), document_variable_name='context'), question_generator=LLMChain(prompt=PromptTemplate(input_variables=[